https://keras.io/preprocessing/image/

# TensorFlow 2.0

In [1]:
import os
import glob

import tensorflow as tf
from tensorflow.keras import layers, datasets
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [3]:
train_dir = './dataset/mnist_png/training'
test_dir  = './dataset/mnist_png/testing'

## Hyperparameter Tunning

In [4]:
num_epochs = 5
batch_size = 32

learning_rate = 0.001
dropout_rate = 0.5

input_shape = (28, 28, 1)
num_classes = 10

## Preprocess

In [5]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   width_shift_range=0.3,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

In [6]:
train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=input_shape[:2],
        batch_size=batch_size,
        color_mode='grayscale'
)

validation_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=input_shape[:2],
        batch_size=batch_size,
        color_mode='grayscale'
)

Found 60000 images belonging to 10 classes.
Found 10000 images belonging to 10 classes.


In [7]:
a, b = next(train_generator)
a.shape, b.shape

((32, 28, 28, 1), (32, 10))

## Build Model

In [8]:
inputs = layers.Input(input_shape)

net = layers.Conv2D(32, (3, 3), padding='SAME')(inputs)
net = layers.Activation('relu')(net)
net = layers.Conv2D(32, (3, 3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.MaxPooling2D(pool_size=(2, 2))(net)
net = layers.Dropout(dropout_rate)(net)

net = layers.Conv2D(64, (3, 3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.Conv2D(64, (3, 3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.MaxPooling2D(pool_size=(2, 2))(net)
net = layers.Dropout(dropout_rate)(net)

net = layers.Flatten()(net)
net = layers.Dense(512)(net)
net = layers.Activation('relu')(net)
net = layers.Dropout(dropout_rate)(net)
net = layers.Dense(num_classes)(net)
net = layers.Activation('softmax')(net)

model = tf.keras.Model(inputs=inputs, outputs=net, name='Basic_CNN')

In [9]:
# Model is the full model w/o custom layers
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

## Training

In [10]:
model.fit(train_generator, 
          validation_data=validation_generator, 
          epochs=num_epochs,
          verbose=2)

Epoch 1/5
1875/1875 - 30s - loss: 0.5475 - accuracy: 0.8178 - val_loss: 0.0890 - val_accuracy: 0.9712
Epoch 2/5
1875/1875 - 30s - loss: 0.2210 - accuracy: 0.9298 - val_loss: 0.0741 - val_accuracy: 0.9755
Epoch 3/5
1875/1875 - 30s - loss: 0.1743 - accuracy: 0.9462 - val_loss: 0.0750 - val_accuracy: 0.9768
Epoch 4/5
1875/1875 - 30s - loss: 0.1558 - accuracy: 0.9529 - val_loss: 0.0583 - val_accuracy: 0.9803
Epoch 5/5
1875/1875 - 30s - loss: 0.1400 - accuracy: 0.9564 - val_loss: 0.0668 - val_accuracy: 0.9778
